Introduction to Natural Language Processing
* Getting the necessary doc and files

In [ ]:
# !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2024-12-11 20:30:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.212.123, 142.250.200.251, 172.217.18.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.212.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: 'nlp_getting_started.zip'

     0K .......... .......... .......... .......... ..........  8%  145K 4s
    50K .......... .......... .......... .......... .......... 16%  280K 3s
   100K .......... .......... .......... .......... .......... 25%  531K 2s
   150K .......... .......... .......... .......... .......... 33%  559K 1s
   200K .......... .......... .......... .......... .......... 42%  721K 1s
   250K .......... .......... .......... .......... .......... 50%  667K 1s
   300K .......... .......... .......... .......... .......... 59%  381K 1s
   350K .......... .......... ..........

In [3]:
from helper_functions import unzip_data, plot_loss_curves, create_tensorboard_callback, compare_historys
unzip_data('nlp_getting_started.zip')

In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'train.csv')
test_data = pd.read_csv(r'test.csv')
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [2]:
# shuffling the train_data
train_data = train_data.sample(frac=1, random_state=42)
train_data.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [3]:
train_data.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [4]:
len(train_data), len(test_data)

(7613, 3263)

In [5]:
import random
random_index = random.randint(0, len(train_data)-5)

for row in train_data[['text', 'target']][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target :{target} ", "(real diaster)" if target>0 else "(not real diseater)")
    print(f"Text :{text} \n")
    print('......\n')

Target :0  (not real diseater)
Text :@accionempresa The U.S. Department of Commerce has been deluged the last two months with com... http://t.co/V1SFlLOWGh @gerenciatodos å¨ 

......

Target :0  (not real diseater)
Text :Fear is the mind killer. Fear is the little-death that brings total obliteration.  Bene Gesserit Litany Against Fear Dune   @atgrannyshouse 

......

Target :0  (not real diseater)
Text :@Toocodtodd Hey @wyattb23 let's challenge then to a tornado tag tlc match. Winner take all. 

......

Target :0  (not real diseater)
Text :@Jolly_Jinu you said they are terrorist because of #Babri so was it ok? If you demolish my house todayhave i right to take revenge? 

......

Target :1  (real diaster)
Text :Vacation update: my great aunt just killed a spider with her bare hand after I screamed in fear. #fierce 

......



Spliting data into training, testing anad validation sets

In [6]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data['text'].to_numpy(), 
                                                                            train_data['target'].to_numpy(),
                                                                            test_size=0.1, 
                                                                            random_state=42) 

In [7]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [8]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

**converting text to numbers**
* Tokenization: direct mapping of tekens(a token could be a word or a character)
* Embedding: creating a metrix of feature vectors for each token(s)(the size of the feature vector can be define and the embedding can be learned)

In [9]:
# Tokenization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [10]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams= None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = False)

In [11]:
len(train_sentences[0].split())

7

In [12]:
# findinf the number of avg word in ine training sentences
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
# setting up text vectorizer
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [14]:
text_vectorizer.adapt(train_sentences)

In [15]:
# creating a smaple sentence tokens

sentence = "This is a sample sentence statement"
text_vectorizer([sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  19,    9,    3, 8839,    1, 2283,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [16]:
# choosing the random sentence in the training set
random_sentence = random.choice(train_sentences)
print(f"Origin text: \n{random_sentence}\n")

text_vectorizer([random_sentence])

Origin text: 
@schelbertgeorg Thanks. I'm teaching an online class &amp; asking my students lots of questions like this. Sorry for the deluge of Ren. art!



<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[8760,  565,   32, 3349,   39,  812,  784,   35, 1900,   13, 1466,
        1504,    6, 2012,   25]], dtype=int64)>

In [17]:
# getting the unique characters in the vocabulary
word_in_vocab = text_vectorizer.get_vocabulary()
top_5 = word_in_vocab[:5]
buttom_5 = word_in_vocab[-5:]
print(len(word_in_vocab))
print(top_5)
print(buttom_5)

10000
['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [18]:
# Setting the embedding space 
from tensorflow.keras.layers import Embedding
embedding = Embedding(
    input_dim = max_vocab_length,
    input_length = max_length,
    output_dim = 128
)

c:\Users\Phavour\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [19]:
# getting random number of embedding
print(f"Original sentences :\n{random_sentence}\n")
print(f"Embedded sentences : \n")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original sentences :
@schelbertgeorg Thanks. I'm teaching an online class &amp; asking my students lots of questions like this. Sorry for the deluge of Ren. art!

Embedded sentences : 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04238025, -0.04673641, -0.03894365, ..., -0.03007999,
          0.04640546, -0.01654596],
        [-0.01239238, -0.02331423, -0.01142812, ...,  0.02253142,
          0.01151562, -0.00837138],
        [-0.04205942,  0.02799735, -0.03296186, ..., -0.01895843,
         -0.00149179,  0.01985994],
        ...,
        [-0.02287855, -0.01463386,  0.04557813, ..., -0.01648057,
          0.02952877, -0.02642081],
        [ 0.01650831,  0.03844723, -0.02046695, ..., -0.01577802,
          0.04351462,  0.0116508 ],
        [-0.00263903,  0.00279985,  0.03263814, ..., -0.03937676,
         -0.04439098, -0.01182207]]], dtype=float32)>

In [20]:
# checking out a single token
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.04238025, -0.04673641, -0.03894365,  0.01927422,  0.02717547,
       -0.04509121,  0.04133408, -0.00731117, -0.04456056, -0.00295416,
        0.0126434 ,  0.00186737,  0.02165482,  0.04136313, -0.0119563 ,
        0.02661412,  0.00630896,  0.0468776 , -0.03630915,  0.02005608,
       -0.01008416,  0.03613765, -0.01818118, -0.01134068, -0.04546743,
       -0.00089904,  0.03123428, -0.02101111,  0.01655003, -0.01852628,
       -0.04965214, -0.02130979,  0.00873418, -0.01939201,  0.02096346,
       -0.04754629, -0.03541823, -0.03595517, -0.04298422,  0.0296106 ,
       -0.02533942,  0.01695044, -0.01018803, -0.02893549, -0.00773687,
       -0.04144064, -0.01181935, -0.02384464,  0.00258396, -0.0248771 ,
        0.03751415,  0.00254403,  0.00492521,  0.04376043, -0.0197094 ,
        0.02902487,  0.0084898 , -0.00707581, -0.04260187, -0.03466081,
       -0.02118336,  0.02147486, -0.02143513,  0.00314426,  0.00488814,
        0.013129

Modelling text datase(running a series of expperiment)
* First Baseline

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [22]:

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# fitting the model
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [23]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Model 0 score: {baseline_score*100:.2f}")

Model 0 score: 79.27


In [24]:
baseline_predictions = model_0.predict(val_sentences)
baseline_predictions[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0], dtype=int64)

In [25]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

**creating a function to keep track of our model accuracies**
* F1 scores
* precision
* accuracy
* recall


In [26]:
# from sklearn.metrics import accuracy_score, recall, f1_score, precision
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_accuracy(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred)*100
    model_precision, model_recall, model_f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    model_results = {
        "model accuracy": model_accuracy,
        "model precision": model_precision*100,
        "model recall": model_recall*100,
        "model f1 score": model_f1_score*100
    }

    return model_results



In [27]:
calculate_accuracy(val_labels, baseline_predictions)

{'model accuracy': 79.26509186351706,
 'model precision': 81.11390004213173,
 'model recall': 79.26509186351706,
 'model f1 score': 78.6218975804955}

model 2 Dense model(Feed forward Neural Network)

In [28]:
logs_dir = 'model logs'

In [44]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, output, name='model_2')

model_2.summary()

Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
from helper_functions import create_tensorboard_callback
model_2.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)



In [46]:
train_sentences.shape, train_labels.shape, val_sentences.shape, val_labels.shape

((6851,), (6851,), (762,), (762,))

In [47]:
model_2_history = model_2.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name = 'model_2')]
)

Saving TensorBoard log files to: model logs/model_2/20241212-232734
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.6530 - loss: 0.6498 - val_accuracy: 0.7546 - val_loss: 0.5350
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8112 - loss: 0.4635 - val_accuracy: 0.7887 - val_loss: 0.4723
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8652 - loss: 0.3529 - val_accuracy: 0.7900 - val_loss: 0.4601
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8937 - loss: 0.2837 - val_accuracy: 0.7927 - val_loss: 0.4663
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9151 - loss: 0.2368 - val_accuracy: 0.7848 - val_loss: 0.4821


In [48]:
model_2.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7728 - loss: 0.5165


[0.4821077287197113, 0.7847769260406494]

In [49]:
model_2_pred_prop = model_2.predict(val_sentences)
model_2_pred_prop.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


(762, 1)

In [50]:
model_2_pred_prop[:10]

array([[0.3444622 ],
       [0.8439178 ],
       [0.9978648 ],
       [0.0861445 ],
       [0.11814766],
       [0.9334937 ],
       [0.926746  ],
       [0.99346554],
       [0.96581006],
       [0.18648234]], dtype=float32)

In [51]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

In [52]:
# model prediction propability to labels
model_pred = tf.squeeze(tf.round(model_2_pred_prop))
model_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>